In [1]:
import argparse
import gc
import logging
import os
import sys
import time

from collections import defaultdict

import torch
import torch.nn as nn
import torch.optim as optim

from sgan.data.loader import data_loader
from sgan.losses_argo import gan_g_loss, gan_d_loss, l2_loss
from sgan.losses_argo import displacement_error, final_displacement_error

from sgan.models_argo import TrajectoryGenerator, TrajectoryDiscriminator
from sgan.utils import int_tuple, bool_flag, get_total_norm
from sgan.utils import relative_to_abs, get_dset_path

from sgan.data.data import Argoverse_Social_Data, collate_traj_social
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

import numpy as np
torch.backends.cudnn.benchmark = True

from matplotlib import pyplot as plt

from matplotlib import pyplot as plt
from argoverse.map_representation.map_api import ArgoverseMap
from argoverse.data_loading.argoverse_forecasting_loader import ArgoverseForecastingLoader
from argoverse.visualization.visualize_sequences import viz_sequence
avm = ArgoverseMap()

In [2]:
def init_weights(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        nn.init.kaiming_normal_(m.weight)
        
def get_dtypes(use_gpu=True):
    long_dtype = torch.LongTensor
    float_dtype = torch.FloatTensor
    if use_gpu == 1:
        long_dtype = torch.cuda.LongTensor
        float_dtype = torch.cuda.FloatTensor
    return long_dtype, float_dtype        

In [3]:
def relativeToAbsolute(traj, R, t):    
    seq_len = traj.shape[0] 
    traj = np.reshape(traj, (-1, 2))
    
    traj = np.swapaxes(traj, 1, 0)
    traj = np.matmul(R.T,traj)
    traj = np.swapaxes(traj, 1, 0)
    traj = traj + t
    traj = np.reshape(traj, (seq_len, -1, 2))
    return traj

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
long_dtype, float_dtype = get_dtypes()

argoverse_val   = Argoverse_Social_Data('../../deep_prediction/data/val/data')
argoverse_test  = Argoverse_Social_Data('../../deep_prediction/data/test_obs/data')
# argoverse_forecasting_loader = ArgoverseForecastingLoader('../../deep_prediction/data/test_obs/data') #simply change to your local path of the data
sample_data  = Argoverse_Social_Data('../forecasting_sample/data')

In [5]:
# for i in range(1000):
#     if argoverse_forecasting_loader[i].agent_traj.shape[0]!=20:
#         print('Not 20')

In [6]:
test_size=len(argoverse_test)
# for i in range(1000):
#     if argoverse_test[i]["train_agent"].shape[0]!=20:
#         print('Not 20')

In [7]:
batch_size = 32
val_loader = DataLoader(argoverse_val, batch_size=batch_size,
                    shuffle=False, num_workers=2,collate_fn=collate_traj_social)
sample_loader = DataLoader(sample_data, batch_size=batch_size,
                shuffle=True, num_workers=2,collate_fn=collate_traj_social)

test_loader = DataLoader(argoverse_test, batch_size=1,
                shuffle=False, num_workers=2,collate_fn=collate_traj_social)

In [8]:
generator = TrajectoryGenerator(
    obs_len=20,
    pred_len=30,
    embedding_dim=16,
    encoder_h_dim=32,
    decoder_h_dim=32,
    mlp_dim=64,
    num_layers=1,
    noise_dim=(8, ),
    noise_type='gaussian',
    noise_mix_type='global',
    pooling_type='pool_net',
    pool_every_timestep=0,
    dropout=0,
    bottleneck_dim=32,
    neighborhood_size=2.0,
    grid_size=8,
    batch_norm=False)

generator.apply(init_weights)
generator.type(float_dtype).train()

TrajectoryGenerator(
  (encoder): Encoder(
    (encoder): LSTM(16, 32)
    (spatial_embedding): Linear(in_features=2, out_features=16, bias=True)
  )
  (decoder): Decoder(
    (decoder): LSTM(16, 32)
    (spatial_embedding): Linear(in_features=2, out_features=16, bias=True)
    (hidden2pos): Linear(in_features=32, out_features=2, bias=True)
  )
  (pool_net): PoolHiddenNet(
    (spatial_embedding): Linear(in_features=2, out_features=16, bias=True)
    (mlp_pre_pool): Sequential(
      (0): Linear(in_features=48, out_features=512, bias=True)
      (1): ReLU()
      (2): Linear(in_features=512, out_features=32, bias=True)
      (3): ReLU()
    )
  )
  (mlp_decoder_context): Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=24, bias=True)
    (3): ReLU()
  )
)

In [9]:
output_dir = '../saved_model/07-05-2020::19:37:23/'
checkpoint_name = 'gan_test'
restore_path = os.path.join(output_dir,'%s_with_model.pt' % checkpoint_name)

checkpoint = torch.load(restore_path)
generator.load_state_dict(checkpoint['g_state'])

<All keys matched successfully>

In [10]:
def poly_fit(traj, traj_len=30, threshold=0.002):
    """
    Input:
    - traj: Numpy array of shape (2, traj_len)
    - traj_len: Len of trajectory
    - threshold: Minimum error to be considered for non linear traj
    Output:
    - int: 1 -> Non Linear 0-> Linear
    """
    t = np.linspace(0, traj_len - 1, traj_len)
    res_x = np.polyfit(t, traj[:, 0], 2, full=True)[1]
    res_y = np.polyfit(t, traj[:, 1], 2, full=True)[1]
    
    if res_x + res_y >= threshold:
        return 1.0
    else:
        return 0.0

In [11]:
def predict_single_traj(train_agent, neighbour, av, seq_path, seq_id, Rs, ts):

    obs_traj = train_agent[0].unsqueeze(0)
    obs_traj = torch.cat((obs_traj, av[0].unsqueeze(0)),0)
    obs_traj = torch.cat((obs_traj, neighbour[0]),0)    
    
    ped_count = obs_traj.shape[0]
    seq_start_end = [[0, ped_count]] # last number excluded
    
    for i in range(1, len(neighbour)):
        obs_traj = torch.cat((obs_traj, train_agent[i].unsqueeze(0)), 0)
        obs_traj = torch.cat((obs_traj, av[i].unsqueeze(0)),0)
        obs_traj = torch.cat((obs_traj, neighbour[i]), 0) 
        
        seq_start_end.append([ped_count, obs_traj.shape[0]])

        num_peds_considered = obs_traj.shape[0] - ped_count
        ped_count = obs_traj.shape[0]

    obs_traj_rel = torch.zeros(obs_traj.shape)
    obs_traj_rel[:,1:,:] = obs_traj[:,1:,:] -  obs_traj[:,:-1,:]    
    seq_start_end = torch.tensor(seq_start_end)

    obs_traj = obs_traj.transpose_(0,1).cuda() 
    obs_traj_rel = obs_traj_rel.transpose_(0,1).cuda() 
    
    pred_traj_fake_rel = generator(obs_traj, obs_traj_rel, seq_start_end)
    pred_traj_fake = relative_to_abs(pred_traj_fake_rel, obs_traj[-1])
        
    pred_traj = torch.cat((obs_traj, pred_traj_fake), 0 )
    pred_traj = pred_traj.cpu().detach().numpy()
    
    pred_agv_traj = np.swapaxes(pred_traj,0,1)[0]
    
        
    return pred_agv_traj

In [12]:
# dataiter = iter(test_loader)
# output_all = {}
# counter = 0
# batch = dataiter.next()

# train_agent = batch['train_agent']
# neighbour = batch['neighbour']
# av = batch['av']
# seq_path = batch['seq_path']
# seq_id = batch['indexes']
# Rs = batch['rotation']
# ts = batch['translation']

# predict_traj = predict_single_traj(train_agent, neighbour, av, seq_path, seq_id, Rs, ts)

# print(predict_traj.shape)


In [13]:
# ax = plt.gca()
# plt.plot(predict_traj[0,0],predict_traj[0,1],'-o',c='r') #starting point here
# plt.plot(predict_traj[:21,0],predict_traj[:21,1],'-',c='b')
# plt.plot(predict_traj[20:,0],predict_traj[20:,1],'-',c='r')

# plt.xlabel('map_x_coord (m)')
# plt.ylabel('map_y_coord (m)')
# ax.set_aspect('equal')
# plt.show()

In [14]:
# predict_traj = get_multiple_forecasts()

In [15]:
# ax = plt.gca()

# plt.plot(predict_traj[0,0,0],predict_traj[0,0,1],'-o',c='r') #starting point here
# plt.plot(predict_traj[0,:21,0],predict_traj[0,:21,1],'-',c='b')

# for i in range(len(predict_traj)):
#     plt.plot(predict_traj[i,20:,0],predict_traj[i,20:,1],'-',c=np.random.rand(3,))
# plt.xlabel('map_x_coord (m)')
# plt.ylabel('map_y_coord (m)')
# ax.set_aspect('equal')
# plt.show()

In [16]:
# def get_multiple_forecasts():
    
#     batch = dataiter.next()
#     print(batch)
#     train_agent = batch['train_agent']
#     neighbour = batch['neighbour']
#     av = batch['av']
#     seq_path = batch['seq_path']
#     seq_id = batch['indexes']
#     Rs = batch['rotation']
#     ts = batch['translation']

#     predict_traj = []
#     for i in range(9,3,-1):
#         predict_traj.append(predict_single_traj(train_agent, neighbour, av, seq_path, seq_id, Rs, ts))
                
#     return int(seq_id[0]), np.array(predict_traj)

In [ ]:
dataiter = iter(test_loader)
output_all = {}
counter = 1

test_size=len(argoverse_test)


for test_i in range (test_size):
    batch = dataiter.next()
    train_agent = batch['train_agent']
    neighbour = batch['neighbour']
    av = batch['av']
    seq_path = batch['seq_path']
    seq_id = batch['indexes']
    Rs = batch['rotation']
    ts = batch['translation']
    
    predict_traj = []
    for i in range(9,3,-1):
        predict_traj.append(predict_single_traj(train_agent, neighbour, av, seq_path, seq_id, Rs, ts))
        
    predict_traj = np.array(predict_traj)
    idx = int(seq_path[0][41:-4])
    output_all[idx] = predict_traj[:,20:50,:]
    counter += 1
    print(seq_id)
    
#     if counter>1000:
#         break
    
    

[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
[10]
[11]
[12]
[13]
[14]
[15]
[16]
[17]
[18]
[19]
[20]
[21]
[22]
[23]
[24]
[25]
[26]
[27]
[28]
[29]
[30]
[31]
[32]
[33]
[34]
[35]
[36]
[37]
[38]
[39]
[40]
[41]
[42]
[43]
[44]
[45]
[46]
[47]
[48]
[49]
[50]
[51]
[52]
[53]
[54]
[55]
[56]
[57]
[58]
[59]
[60]
[61]
[62]
[63]
[64]
[65]
[66]
[67]
[68]
[69]
[70]
[71]
[72]
[73]
[74]
[75]
[76]
[77]
[78]
[79]
[80]
[81]
[82]
[83]
[84]
[85]
[86]
[87]
[88]
[89]
[90]
[91]
[92]
[93]
[94]
[95]
[96]
[97]
[98]
[99]
[100]
[101]
[102]
[103]
[104]
[105]
[106]
[107]
[108]
[109]
[110]


In [ ]:
int(seq_path[0][41:-4])

In [ ]:
# from argoverse.evaluation.competition_util import generate_forecasting_h5

# len(output_all)

# output_path = '../../sgan/scripts/new_eval/'

# generate_forecasting_h5(output_all, output_path) #this might take awhile

In [ ]:
key_storage=output_all.keys()

In [ ]:
key_storage=list(key_storage)

In [ ]:
key_storage.sort()

In [ ]:
key_storage[999]

In [ ]:
output_all[80599].shape